In [1]:
'''40. 係り受け解析結果の読み込み（形態素）Permalink
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）
をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，
各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ'''

filename = 'data/ch5/neko.txt.cabocha'

class Morph():
    def __init__(self, line):
        surface, details = line.rstrip().split('\t')
        details = details.split(',')
        base, pos, pos1 = details[6], details[0], details[1]
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(self.surface, self.base, self.pos, self.pos1)
    
    
def read_cabocha(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
        sentence = []
        for line in f:
            if line == "EOS\n":
                if len(sentence):
                    yield sentence
                sentence = []
            else:
                if line[0] == '*':
                    continue
                sentence.append(Morph(line))

for i, morphs in enumerate(read_cabocha(filename)):
    if i == 4:
        for morph in morphs:
            print(morph)

surface[吾輩]	base[吾輩]	pos[名詞]	pos1[代名詞]
surface[は]	base[は]	pos[助詞]	pos1[係助詞]
surface[ここ]	base[ここ]	pos[名詞]	pos1[代名詞]
surface[で]	base[で]	pos[助詞]	pos1[格助詞]
surface[始め]	base[始める]	pos[動詞]	pos1[自立]
surface[て]	base[て]	pos[助詞]	pos1[接続助詞]
surface[人間]	base[人間]	pos[名詞]	pos1[一般]
surface[という]	base[という]	pos[助詞]	pos1[格助詞]
surface[もの]	base[もの]	pos[名詞]	pos1[非自立]
surface[を]	base[を]	pos[助詞]	pos1[格助詞]
surface[見]	base[見る]	pos[動詞]	pos1[自立]
surface[た]	base[た]	pos[助動詞]	pos1[*]
surface[。]	base[。]	pos[記号]	pos1[句点]


In [2]:
'''41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．
このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），
係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．
さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，
8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．'''
from collections import OrderedDict

  
class Chunk():
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
    def show_surface(self):
        return "".join([morph.surface for morph in self.morphs])  
    def show_dst(self):
        return self.dst 
    def show_srcs(self):
        return self.srcs
        
def read_cabocha(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
        idx = -1
        dsts = []
        chunks = []
        for i,line in enumerate(f):
            # if EOS appears, this sentence is end
            if line == "EOS\n":
                if len(chunks):
                    for index,d in enumerate(dsts):
                        chunks[d].srcs.append(index)
                    yield chunks
                    dsts = []
                    chunks = []
                    
            # parsing result, store index and dst at this point
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(cols[2][:-1])
                
                chunks.append(Chunk())
                chunks[-1].dst = dst

                if dst!=-1:  # if dst=-1, means end of section
                    dsts.append(dst)                     
                    
            # we have to store all stuff after parsing result appeared.
            else:
                chunks[-1].morphs.append(Morph(line))
                

for i, chunks in enumerate(read_cabocha(filename)):
    if i == 3:
        for j, chunk in enumerate(chunks):
            print("index:[{}]{}\tsrc:{}\tdst: {:<5}".format(j, chunk.show_surface(),chunk.show_srcs(),chunk.show_dst()))
        break

index:[0]何でも	src:[]	dst: 1    
index:[1]薄暗い	src:[0]	dst: 3    
index:[2]じめじめした	src:[]	dst: 3    
index:[3]所で	src:[1, 2]	dst: 5    
index:[4]ニャーニャー	src:[]	dst: 5    
index:[5]泣いて	src:[3, 4]	dst: 7    
index:[6]いた事だけは	src:[]	dst: 7    
index:[7]記憶している。	src:[5, 6]	dst: -1   


In [3]:
'''42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．
ただし，句読点などの記号は出力しないようにせよ．'''
        
for i,chunks in enumerate(read_cabocha(filename)):
    if i==10: break
    for chunk in chunks:
        if chunk.dst != -1:
            src = chunk.show_surface()  
            dst = chunks[chunk.dst].show_surface() # dependency 
            print('{}\t{}'.format(src, dst))
    print('-'*80)


　	猫である。
吾輩は	猫である。
--------------------------------------------------------------------------------
名前は	無い。
まだ	無い。
--------------------------------------------------------------------------------
　どこで	生れたか
生れたか	つかぬ。
とんと	つかぬ。
見当が	つかぬ。
--------------------------------------------------------------------------------
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している。
いた事だけは	記憶している。
--------------------------------------------------------------------------------
吾輩は	見た。
ここで	始めて
始めて	人間という
人間という	ものを
ものを	見た。
--------------------------------------------------------------------------------
しかも	種族であったそうだ。
あとで	聞くと
聞くと	種族であったそうだ。
それは	種族であったそうだ。
書生という	人間中で
人間中で	種族であったそうだ。
一番	獰悪な
獰悪な	種族であったそうだ。
--------------------------------------------------------------------------------
この	書生というのは
書生というのは	話である。
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である。
--------------------------------------------------------------------------------
しかし	思わなかった。
その	当時は
当時は	なかったから
何という	考も
考も	なかったから
なかったから	思わなかった。
別段	恐し
恐し	思わなかった

In [7]:
'''43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．
ただし，句読点などの記号は出力しないようにせよ．'''

    
class Chunk():
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
    def show_surface(self):
        return "".join([morph.surface for morph in self.morphs])  
    def show_dst(self):
        return self.dst 
    def show_srcs(self):
        return self.srcs  
    def check_pos(self, specified):
        for morph in self.morphs:
            if morph.pos == specified:
                return True
        return False    

for i,chunks in enumerate(read_cabocha(filename)):
    if i==10: break
    for chunk in chunks:
        if chunk.dst != -1:
            # contains noun and dependency section is verb
            if chunk.check_pos("名詞") and chunks[chunk.dst].check_pos("動詞"):
                src = chunk.show_surface()
                dst = chunks[chunk.dst].show_surface()
                print('{}\t{}'.format(src, dst))

　どこで	生れたか
見当が	つかぬ。
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している。
吾輩は	見た。
ここで	始めて
ものを	見た。
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである。
上で	落ちついて
顔を	見たのが
ものの	見始であろう。


In [72]:
#-*- coding: utf-8 -*-
# !pip install pydot
# !apt-get install graphviz  --yes
# !conda install graphviz --yes

'''44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．
可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．
また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．'''

import pydot

for i, chunks in enumerate(read_cabocha(filename)):
    if i == 2:
        target = chunks
        break

pairs = []
for morph in target:
    if int(morph.dst) != -1:
        preText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in morph.morphs])
        postText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in target[int(morph.dst)].morphs])
        pairs.append([preText, postText])
        
# pairs = [['dokode','umaretaka'],['umaretaka','tsukanu'],['tonnto','tsukanu'],['miatariga','tsukanu']]

g = pydot.graph_from_edges(pairs,  directed=True)
print(g)
# g.write_png('data/ch5/ans44.png')

digraph G {
"どこで" -> "生れたか";
"生れたか" -> "つかぬ";
"とんと" -> "つかぬ";
"見当が" -> "つかぬ";
}



In [8]:
'''45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい．
動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える．
この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，
「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．
始める  で
見る    は を'''

class Chunk():
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
        
    def show_surface(self):
        return "".join([morph.surface for morph in self.morphs])  
    
    def show_dst(self):
        return self.dst 
    
    def show_srcs(self):
        return self.srcs 
    
    def check_pos(self, specified):
        for morph in self.morphs:
            if morph.pos == specified:
                return True
        return False    
    
    def get_morphs_pos(self, pos, pos1=''):
        tars = []
        if len(pos1):
            for morph in self.morphs:
                if morph.pos == pos and morph.pos1 == pos1:
                    tars.append(morph)
        else:
            for morph in self.morphs:
                if morph.pos == pos:
                    tars.append(morph)
        return tars


fname_out = 'data/ch5/neko_aux.txt'
with open(fname_out, mode='w',encoding='utf-8') as f:
    for i,chunks in enumerate(read_cabocha(filename)):
#         if i==3: 
        for chunk in chunks:
            auxs = []            
            verbs = chunk.get_morphs_pos("動詞")
            # if verb exists in the section
            if verbs:
                for src in chunk.srcs:
                    aux = chunks[src].get_morphs_pos("助詞")            
                    if len(aux) > 0:
                        auxs.append(aux[-1])
                if len(auxs) < 1:
                    continue
#                 print("{}\t{}\n".format(verbs[0].base," ".join(sorted([aux.surface for aux in auxs]))))
                f.write("{}\t{}\n".format(verbs[0].base," ".join(sorted([aux.surface for aux in auxs]))))


In [10]:
'''46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ
．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える．
この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，
「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で      ここで
見る    は を   吾輩は ものを'''



fname_out = 'data/ch5/neko_aux.txt'

for i,chunks in enumerate(read_cabocha(filename)):
    if i==10: break 
    for chunk in chunks:
        auxs = []       
        aux_sent = []
        verbs = chunk.get_morphs_pos("動詞")
        # if verb exists in the section
        if verbs:
            for src in chunk.srcs:
                aux = chunks[src].get_morphs_pos("助詞")
                if len(aux) > 0:
                    auxs.append(aux[-1])
                    aux_sent.append(chunks[src].show_surface())
            if len(auxs) < 1:
                continue
            print("{}\t{}\t{}".format(verbs[0].base," ".join(
                sorted([aux.surface for aux in auxs]))," ".join(aux_sent)))


生れる	で	　どこで
つく	か が	生れたか 見当が
泣く	で	所で
する	て は	泣いて いた事だけは
始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て
思う	から	なかったから
載せる	に	掌に
持ち上げる	て と	載せられて スーと
ある	が	感じが
落ちつく	で	上で
見る	て を	落ちついて 顔を
見る	の	ものの


In [11]:
'''47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

返事をする      と に は        及ばんさと 手紙に 主人は
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語（サ変接続名詞+を+動詞）
コーパス中で頻出する述語と助詞パターン'''
    
class Chunk():
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
        
    def show_surface(self):
        return "".join([morph.surface for morph in self.morphs])  
    
    def show_dst(self):
        return self.dst 
    
    def show_srcs(self):
        return self.srcs 
    
    def check_pos(self, specified):
        for morph in self.morphs:
            if morph.pos == specified:
                return True
        return False    
    
    def get_morphs_pos(self, pos, pos1=''):
        tars = []
        if len(pos1):
            for morph in self.morphs:
                if morph.pos == pos and morph.pos1 == pos1:
                    tars.append(morph)
        else:
            for morph in self.morphs:
                if morph.pos == pos:
                    tars.append(morph)
        return tars

    def get_sa_verb(self):
        tars = []    
        for i in range(len(self.morphs[0:-1])):
            if self.morphs[i].pos == '名詞' and self.morphs[i].pos1 == 'サ変接続':
                if self.morphs[i+1].surface == 'を' and self.morphs[i+1].pos == '助詞':
                    tars.append([self.morphs[i].surface + self.morphs[i+1].surface])    
        return tars
    

with open('data/ch5/47.txt','w', encoding='utf-8') as f:
    for i,chunks in enumerate(read_cabocha(filename)):
        for chunk in chunks:
            auxs = []       
            aux_sent = []
            verbs = chunk.get_morphs_pos("動詞")
            if len(verbs) < 1:
                continue

            for src in chunk.srcs:
                aux = chunks[src].get_morphs_pos("助詞")
                if len(aux):
                    auxs.append(aux[-1]) #take the last aux
                    aux_sent.append(chunks[src])
            if len(auxs) < 1:
                continue

            for aux_part in aux_sent:
                sa_verb = aux_part.get_sa_verb()
                if len(sa_verb):
                    aux_sent.remove(aux_sent[-1])
                    auxs.remove(auxs[-1])

            if len(sa_verb) < 1:
                continue

            s = sorted(zip([aux.surface for aux in auxs],[sent.show_surface() for sent in aux_sent]), key=lambda x: x[0])
            aux_vs = [aux_v[0] for aux_v in s]
            aux_sent = [aux_v[1] for aux_v in s]

    #         print("{} {} {}\n".format("".join(sa_verb[0])+verbs[0].base," ".join(aux_vs)," ".join(aux_sent)))
            f.write("{}\t{}\t{}\n".format("".join(sa_verb[0])+verbs[0].base," ".join(aux_vs)," ".join(aux_sent)))


# !cut --fields=1 data/ch5/47.txt | sort | uniq --count | sort --numeric-sort --reverse >  data/ch5/47_col1.txt
# !cut --fields=1,2 data/ch5/47.txt | sort | uniq --count | sort --numeric-sort --reverse >  data/ch5/47_col12.txt

In [42]:
'''48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た'''
      
for i,chunks in enumerate(read_cabocha(filename)):
    if i==5: break
    for chunk in chunks:
        noun = chunk.get_morphs_pos("名詞")
        if len(noun):
            dst = chunk.dst
            print(chunk.show_surface(),end='')
            while dst!=-1:
                print('->',chunks[dst].show_surface(),end='')
                dst = chunks[dst].dst
            print('\n')

吾輩は-> 猫である。

猫である。

名前は-> 無い。

　どこで-> 生れたか-> つかぬ。

見当が-> つかぬ。

何でも-> 薄暗い-> 所で-> 泣いて-> 記憶している。

所で-> 泣いて-> 記憶している。

ニャーニャー-> 泣いて-> 記憶している。

いた事だけは-> 記憶している。

記憶している。

吾輩は-> 見た。

ここで-> 始めて-> 人間という-> ものを-> 見た。

人間という-> ものを-> 見た。

ものを-> 見た。



In [41]:
'''49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．
ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
文節iとjに含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 
    文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示
例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y'''

'''48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た'''


class Chunk():
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
        
    def show_surface(self):
        return "".join([morph.surface for morph in self.morphs])  
    
    def show_dst(self):
        return self.dst 
    
    def show_srcs(self):
        return self.srcs 
    
    def check_pos(self, specified):
        for morph in self.morphs:
            if morph.pos == specified:
                return True
        return False    
    
    def get_morphs_pos(self, pos, pos1=''):
        tars = []
        if len(pos1):
            for morph in self.morphs:
                if morph.pos == pos and morph.pos1 == pos1:
                    tars.append(morph)
        else:
            for morph in self.morphs:
                if morph.pos == pos:
                    tars.append(morph)
        return tars
    
    def convert_symbol(self, symbol):
        result = []
        for morphs in self.morphs:
            if morphs.pos == "名詞" and morphs.pos!='記号':
                result += symbol
            else:
                result += morphs.surface
        return "".join(result)
    
def collision_check(chunks, noun1, noun2):
    route1 = [noun1]
    route2 = [noun2]

    dst = noun1.dst
    while dst!=-1:
        route1.append(chunks[dst])
        dst = chunks[dst].dst

    dst = noun2.dst        
    while dst!=-1:
        route2.append(chunks[dst])
        dst = chunks[dst].dst    

    if noun2 in route1:
        return -1
    else:
        s = [c.dst for c in route2 if chunks[c.dst] in route1][0]
        return s    

def print_path(path, chunks, noun, symbol, collapse):
    print(noun.convert_symbol(symbol),end='')
    dst = noun.dst
    if path =='y2x':
        while dst != collapse:
            print(' -> ' + chunks[dst].show_surface(),end='')
            dst = chunks[dst].dst
        print(' | ',end='')   
    else:
        while dst != -1:   
            if chunks[dst] == collapse:
                result = chunks[dst].convert_symbol('Y')
                print(' -> ' + result[:result.index('Y') + 1])
                break
            print(' -> ' +  chunks[dst].show_surface(),end='')   
            dst = chunks[dst].dst         

def get_path(chunks):
    nouns = [chunk for chunk in chunks if "名詞" in [m.pos for m in chunk.morphs]]     
    if len(nouns) > 1:
        for i,noun1 in enumerate(nouns[:-1]):
            for j,noun2 in enumerate(nouns[i+1:]):
                collapse = collision_check(chunks, noun1, noun2)

                if collapse == -1:
                    print_path('x2y', chunks, noun1, 'X', noun2)
                    
                else:
                    print_path('y2x', chunks, noun1, 'X',collapse)
                    print_path('y2x', chunks, noun2, 'Y',collapse)
                    print(chunks[collapse].show_surface())
    
filename = 'data/ch5/neko.txt.cabocha'
for i,chunks in enumerate(read_cabocha(filename)):
    if i==4: 
        get_path(chunks)

Xは | Yで -> 始めて -> 人間という -> ものを | 見た。
Xは | Yという -> ものを | 見た。
Xは | Yを | 見た。
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
